<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/SVM%26RF_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report


import os
os.chdir('drive/My Drive/MCA Dataset')

import nltk
nltk.download('stopwords')

# Preparing Data

In [0]:
dataset1 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))
pos = []
neg = []

countPos = 0

def checkPosNeg(dataset, index):
    for i in range(0, len(dataset)):
        if(dataset[i][0] == index):
            return dataset[i][1]
    return 0

for i in range(len(dataset)):
    if dataset[i][0]<=310 and dataset[i][0]>=300:
        if(dataset[i][1] == 1):
            neg.append(dataset[i][0])
        else:
            pos.append(dataset[i][0])
    elif dataset[i][0]<=355 and dataset[i][0]>=345:
        if(dataset[i][1] == 1):
            neg.append(dataset[i][0])
        else:
            pos.append(dataset[i][0])
            
pos = np.array(pos)
neg = np.array(neg)

Data = []
Y = []

Data_test = []
Y_test = []
index = -1
for i in range(0, len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    if(val == 0 and countPos>37):
        continue
    Y.append(val)
    index+=1
    if(Y[index] == 0):
        countPos+=1

    
    fileName = "/content/drive/My Drive/MCA Dataset/train_data/" + str(int(dataset3[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])


for i in range(0, len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    if(val == 0):
        continue
    Y.append(val)
    fileName = "/content/drive/My Drive/MCA Dataset/dev_data/" + str(int(dataset1[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])
    
for i in range(0, len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    fileName = "/content/drive/My Drive/MCA Dataset/test_data/" + str(int(dataset2[i][0])) + "_TRANSCRIPT.csv"
    Data_test.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])

    
Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)
    
for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)
        
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/GoogleNews-vectors-negative300.bin', binary=True)

stop_words = set(stopwords.words('english'))


def remove_StopWOrds(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    return filtered_sentence

def checkAcc(Y_pred, Y_test):
    correct = 0
    for i in range(len(Y_pred)):
        if(Y_pred[i] == Y_test[i]):
            correct+=1
    
    return float(correct)/len(Y_pred)


max_num_words = 15
max_num_sentence = 200

#train_data
finalMatrix = np.zeros((Data2.shape[0], max_num_sentence, max_num_words))
print(finalMatrix.shape)
for k in range(Data2.shape[0]):
    for i in range(min(max_num_sentence, len(Data2[k][0]))):
    	sentence = Data2[k][i].split(" ")
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			finalMatrix[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X = np.zeros((len(Data2), max_num_sentence*max_num_words))
for i in range(len(finalMatrix)):
    X[i] = finalMatrix[i].flatten().reshape(1, -1)
 
#Test_data
finalMatrix_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words))
print(finalMatrix_test.shape)
for k in range(Data2_test.shape[0]):
    for i in range(min(max_num_sentence, len(Data2_test[k][0]))):
    	sentence = Data2_test[k][i].split(" ")
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			finalMatrix_test[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X_test = np.zeros((len(Data2_test), max_num_sentence*max_num_words))
for i in range(len(finalMatrix_test)):
    X_test[i] = finalMatrix_test[i].flatten().reshape(1, -1)

# Running Models

In [0]:
class RfText:
  def __init__(self):
    self.classifier = RandomForestClassifier()
  
  def fitModel(self, X, Y):
    self.classfier.fit(X, Y)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def scoreModel(self, X, Y):
    return self.classifier.score(X, Y)

class SVMText:
  def __init__(self, kernel = "rbf"):
    self.classifier = SVC(kernel = kernel, random_state = 0)
  
  def fitModel(self, X, Y):
    self.classfier.fit(X, Y)
  
  def predictModel(self, X):
    return self.classifier.predict(X)
  
  def scoreModel(self, X, Y):
    return self.classifier.score(X, Y)

print("Random forest: ")
rfModel = rfText()
rfModel.fitModel(X, Y)
Y_pred1 = rfModel.predictModel(X_test)
print(classification_report(Y_test,Y_pred1))

print("SVM: ")
svmModel = SVMText()
svmModel.fitModel(X, Y)
Y_pred2 = svmModel.predictModel(X_test)
print(classification_report(Y_test, Y_pred2))